In [22]:
import requests
import pandas as pd
import urllib.parse
import numpy as np
import re

In [17]:
make = 'audi'
model = 'a3'
yearMin = 2010
yearMax = 2020
kmMin = 0
kmMax = 100000
priceMin = 0
priceMax = 10000

'https://api.wallapop.com/api/v3/cars/search?filters_source=suggester&keywords=audi%20a4&category_ids=100&longitude=-3.69196&latitude=40.41956'
url_base = 'https://api.wallapop.com/api/v3/cars/search?'
parametros = {
    'filters_source': 'suggester',
    'keywords': make + ' ' + model,
    'category_ids': 100,
    'longitude': -3.69196,
    'latitude': 40.41956,
    'yearMin': yearMin,
    'yearMax': yearMax,
    'kmMin': kmMin,
    'kmMax': kmMax,
    'priceMin': priceMin,
    'priceMax': priceMax
}
url_completa = url_base + urllib.parse.urlencode(parametros)
response = requests.get(url_completa)
if response.ok:
    try:
        data = response.json()
        rows = []
        for obj in data['search_objects']:
            row = obj['content']
            rows.append(row)

        df = pd.DataFrame(rows)
    except Exception as e:
        print("Error:", e)


In [21]:
df = df[['title', 'price', 'km', 'engine', 'gearbox', 'horsepower']]

def scrape_year(title):
    # Using regular expression to get a 4-digit combination starting with 1 or 2
    pattern = r'\b(?:1|2)\d{3}\b'
    match = re.search(pattern, title)
    if match:
        year = match.group(0)
        return int(year)
    else:
        return None


df['year'] = df['title'].apply(scrape_year)
df = df.dropna()
df['environmental_label'] = np.nan
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 20
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       10 non-null     object 
 1   price       10 non-null     float64
 2   km          10 non-null     int64  
 3   engine      10 non-null     object 
 4   gearbox     10 non-null     object 
 5   horsepower  10 non-null     float64
 6   year        10 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 640.0+ bytes
